In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:

df = pd.read_csv("database.csv")  

In [ ]:

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        23412 non-null  object 
 1   Time                        23412 non-null  object 
 2   Latitude                    23412 non-null  float64
 3   Longitude                   23412 non-null  float64
 4   Type                        23412 non-null  object 
 5   Depth                       23412 non-null  float64
 6   Depth Error                 4461 non-null   float64
 7   Depth Seismic Stations      7097 non-null   float64
 8   Magnitude                   23412 non-null  float64
 9   Magnitude Type              23409 non-null  object 
 10  Magnitude Error             327 non-null    float64
 11  Magnitude Seismic Stations  2564 non-null   float64
 12  Azimuthal Gap               7299 non-null   float64
 13  Horizontal Distance         160

In [ ]:

df = pd.read_csv("database.csv")  

df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors='coerce')
df.dropna(subset=['DateTime'], inplace=True)  


epoch = datetime(1970, 1, 1)
df['TimeStamp'] = df['DateTime'].apply(lambda x: (x - epoch).total_seconds())


df = df[['TimeStamp', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]

In [ ]:

lat = float(input("Enter Latitude: "))
long = float(input("Enter Longitude: "))
depth = float(input("Enter Depth: "))


location_radius = 1.0
filtered_df = df[(df['Latitude'].between(lat - location_radius, lat + location_radius)) &
                 (df['Longitude'].between(long - location_radius, long + location_radius))]


while filtered_df.empty and location_radius <= 5.0:
    location_radius += 1.0
    filtered_df = df[(df['Latitude'].between(lat - location_radius, lat + location_radius)) &
                     (df['Longitude'].between(long - location_radius, long + location_radius))]


if filtered_df.empty:
    print("No direct data available. Using global average values.")
    avg_lat = df['Latitude'].mean()
    avg_long = df['Longitude'].mean()
    avg_depth = df['Depth'].mean()
    avg_mag = df['Magnitude'].mean()

    filtered_df = df.copy()
else:
    avg_lat, avg_long, avg_depth, avg_mag = lat, long, depth, np.percentile(filtered_df['Magnitude'], 90)


threshold_mag = np.percentile(filtered_df['Magnitude'], 90)
print(f"Automatically determined threshold magnitude: {threshold_mag:.2f}")


filtered_df = filtered_df.sort_values(by='TimeStamp')
filtered_df['TimeDiff'] = filtered_df['TimeStamp'].diff().fillna(0)


Enter Latitude: 11
Enter Longitude: 22
Enter Depth: 30
No direct data available. Using global average values.
Automatically determined threshold magnitude: 6.40


In [ ]:

filtered_df = filtered_df.sort_values(by='TimeStamp')
filtered_df['TimeDiff'] = filtered_df['TimeStamp'].diff().fillna(0)


features = ['TimeStamp', 'Latitude', 'Longitude', 'Depth', 'Magnitude']
X = filtered_df[features].values
y = filtered_df['TimeDiff'].values


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])


train_size = int(0.8 * len(X_scaled))
if train_size == 0:
    print("Not enough data to train the model.")
    exit()

X_train, X_test = X_scaled[:train_size], X_scaled[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(1, X_scaled.shape[2])),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:

model.compile(optimizer='adam', loss='mse', metrics=['mae'])


if len(X_train) > 0:
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
else:
    print("Not enough training data. Try a different location.")
    exit()


input_data = np.array([[datetime.now().timestamp(), avg_lat, avg_long, avg_depth, threshold_mag]])
input_scaled = scaler.transform(input_data).reshape(1, 1, X_scaled.shape[2])
pred_time_diff = model.predict(input_scaled)[0][0]

pred_days = pred_time_diff / (60 * 60 * 24)
print(f"Predicted time until next earthquake ≥ {threshold_mag:.2f} magnitude: {pred_days:.2f} days")

Epoch 1/50
586/586 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 12554711040.0000 - mae: 72057.3516 - val_loss: 8828437504.0000 - val_mae: 58924.0078
Epoch 2/50
586/586 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 11439880192.0000 - mae: 67321.1406 - val_loss: 6950641152.0000 - val_mae: 50821.7031
Epoch 3/50
586/586 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 9908343808.0000 - mae: 59355.9297 - val_loss: 5619497472.0000 - val_mae: 51621.7852
Epoch 4/50
586/586 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 7963151360.0000 - mae: 57531.1445 - val_loss: 5712595456.0000 - val_mae: 57322.5781
Epoch 5/50
586/586 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 7857408000.0000 - mae: 60551.1680 - val_loss: 5828130304.0000 - val_mae: 58986.6875
Epoch 6/50
586/586 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 7752920064.0000 - mae: 60898.9336 - val_loss: 5842983424.0000 - val_mae: 59189.7891
Epoch 7/50
586/586 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 7709515776.0000 - mae: 61005.5859 - val_loss: 5813609984.0000 - val_mae: 5